# Global Terrorism
data source: [https://www.kaggle.com/START-UMD/gtd](https://www.kaggle.com/START-UMD/gtd)
*The Global Terrorism Database (GTD) is an open-source database including information on terrorist attacks around the world from 1970 through 2016*<br>

**Packages neeeded to run this notebook:**
1. numpy
2. pandas
3. plotly

In [17]:
#jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000 

import numpy as np
import pandas as pd
import plotly.plotly as py

import plotly.graph_objs as go
from plotly import tools
from plotly.offline import download_plotlyjs,iplot, init_notebook_mode
import plotly.figure_factory as ff
init_notebook_mode(connected=True)


## Reading data from csv file and renaming columns

In [18]:
data = pd.read_csv('globalterrorismdb_0617dist.csv', encoding = 'ISO-8859-1', usecols = [1,2,8,10,26,29,35,98,101])


data = data.rename(columns={'iyear':'year', 'imonth':'month', 'country_txt':'country', 'region_txt':'region',
                            'attacktype1_txt':'attack_type', 'targtype1_txt':'target_type', 'nkill':'fatalities', 'nwound':'injuries'})

## Number of terrorist attacks over the years

In [20]:
years_attacks = data.groupby(['year']).size().reset_index(name='Attacks')

trace = go.Scatter(
    x = years_attacks['year'],
    y = years_attacks['Attacks'],
)

layout = dict(title='Terrorist attacks over the years',
              xaxis = dict(title = 'Years'),
              yaxis = dict(title = 'Terrorist attacks'),
              )

data_years = [trace]

fig = dict(data=data_years, layout=layout)

iplot(fig, filename='Terrorist attacks over the years')


## Deaths and injuries per attack in countries

In [21]:
country_attacks = data.groupby(['country']).size().to_frame('Attacks')

country_attacks['country']=country_attacks.index

year_country=data.groupby(['year','country']).size().reset_index(name="Attacks")

injuried = data.groupby('country')['injuries'].agg(['sum']).rename(columns={'sum':'injuries'})

#print(injuried)

fatalities = data.groupby('country')['fatalities'].agg(['sum']).rename(columns={'sum':'fatalities'})

#print(fatalities)

injuried_fatalities=pd.concat([fatalities,injuried['injuries'],country_attacks['Attacks'],(fatalities['fatalities']/country_attacks['Attacks']).rename("Deaths/attack"),(injuried['injuries']/country_attacks['Attacks']).rename("Injuries/attack")],axis=1)

injuried_fatalities=injuried_fatalities.rename_axis("country", axis="columns").rename_axis('')

injuried_fatalities['country']=injuried_fatalities.index

#print(injuried_fatalities)

trace0 = go.Scatter(
    x=injuried_fatalities['country'],
    y=injuried_fatalities['Deaths/attack'],
    mode = 'lines',
    name = 'Deaths per attack'
)

trace1 = go.Scatter(
    x=injuried_fatalities['country'],
    y=injuried_fatalities['Injuries/attack'],
    mode = 'lines',
    name = 'Injuries per attack'
)

layout = dict(title='Deaths and injuries per attack in countries',
              xaxis = dict(title = 'Country'),
              yaxis = dict(title = 'People dead/injured'),
              )

data_da = [trace0,trace1]

fig = dict(data=data_da, layout=layout)

iplot(fig, filename='Deaths and injuries per attack in countries')

## Terrorist attacks in countries overall

In [22]:
datac = [ dict(
        type = 'choropleth',
        locationmode = 'country names',
        locations = country_attacks['country'],
        z = country_attacks['Attacks'],
        colorscale = [[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
        [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
               # color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            title = 'Number of attacks'),
      ) ]

layout = dict(
    title = 'Terrorist attacks in countries overall',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=datac, layout=layout )
iplot( fig, validate=True, filename='Terrorist attacks in countries overall' )

## Terrorist attacks in countries over the years

In [23]:
dataoy = [dict(type='choropleth',
            visible = True,
            locationmode = 'country names',
            locations = year_country.loc[year_country['year'] == 1970]['country'],
            z = year_country.loc[year_country['year'] == 1970]['Attacks'],
            colorscale = [[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
            [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
            autocolorscale = False,
            reversescale = True,
            colorbar = dict(
            title = 'Number of attacks'))]
                                        

for i in range(46):
    dataoy.append(dict(type='choropleth',
            visible = False,
            locationmode = 'country names',
            locations = year_country.loc[year_country['year'] == i+1971]['country'],
            z = year_country.loc[year_country['year'] == i+1971]['Attacks'],
            colorscale = [[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
            [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
            autocolorscale = False,
            reversescale = True,
            colorbar = dict(
            title = 'Number of attacks')))
#print(data)

steps = []
for i in range(len(dataoy)):
    step = dict(method='restyle',
                args=['visible', [False] * len(dataoy)],
                label='Year {}'.format(i + 1970))
    step['args'][1][i] = True
    steps.append(step)
    
del steps[23] # no data for year 1993
    
sliders = [dict(active=0,
                pad={"t": 1},
                steps=steps)]    

layout = dict(
        title = 'Terrorist attacks in countries over the years',
        geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(type = 'Mercator')),
        sliders=sliders)
             

fig = dict(data=dataoy, layout=layout)
iplot(fig,filename='Terrorist attacks in countries over the years')

## Most typical attack types per target type

In [24]:
attype_target = data.groupby(['attack_type','target_type']).size().reset_index(name='Attacks').sort_values('Attacks',ascending=False).drop_duplicates(['target_type'])

table = ff.create_table(attype_target)
         
iplot(table, filename='pandas_table')